# Fixed income optimization

## Load data

In [2]:
import pandas as pd
import numpy as np
import cvxpy as cp


In [3]:
def load_matrices(input_sheet, input_tab):
    df = pd.read_excel(input_sheet, sheet_name=input_tab)

    if "sec_id" not in df.columns:
        raise ValueError("The dataframe must have a column named 'sec_id'")
    

    df = df.sort_values(by="sec_id")


    sec_id = df["sec_id"].astype("category")


    sectors = df[[col for col in df.columns if col.startswith("s_")]].to_numpy()


    weight = df[["weight"]].to_numpy()


    dts = df[[col for col in df.columns if col.startswith("dts")]].to_numpy()


    dts = np.nan_to_num(dts)


    quality = df[[col for col in df.columns if col.startswith("q_")]].to_numpy()


    return (sectors, weight, dts, quality, sec_id)


In [4]:
input_sheet = "../data/optimisation-clean_data.xlsx"

sectors_model, weight_model, dts_model, quality_model, sec_id_model = load_matrices(
    input_sheet, "model"
)


assert abs(sum(weight_model) - 1) < 1e-6

sectors_target, weight_target, dts_target, quality_target, sec_id_target = load_matrices(
    input_sheet, "target3"
)


assert list(sec_id_model) == list(sec_id_target)

# Model

Defining variables


In [5]:
# model
S_m = sectors_model
W_m = weight_model
dts_m = dts_model
Q_m = quality_model

# target data
S_t = sectors_target
W_t = weight_target
dts_t = dts_target
Q_t = quality_target


Compute weighted version of sectors, dts, and quality

In [6]:
C_1 = S_m.T @ W_m
C_2 = Q_m.T @ W_m
C_3 = dts_m.T @ W_m


In [7]:
n = sec_id_model.shape[0]

w = cp.Variable((n, 1))

lamba1 = 0.2
lamba2 = 0.2
lamba3 = 0.6

objective = cp.Minimize(
    lamba3 * cp.norm(C_3 - dts_t.T @ w, 2)
    + lamba2 * cp.norm(C_2 - Q_t.T @ w, 2)
    + lamba1 * cp.norm(C_1 - S_t.T @ w, 2)
)

constraints = [w >= 0, w <= 0.03, cp.sum(w) == 1]

prob = cp.Problem(objective, constraints)
prob.solve()

print("status:", prob.status)
print("optimal value", prob.value)
w_opt_1 = w.value

status: optimal
optimal value 0.0006018234877872264


In [8]:
print(Q_m.T @ W_m)

[[0.14580964]
 [0.04662234]
 [0.18196216]
 [0.59571393]
 [0.02989193]]


In [9]:
print(Q_t.T @ w_opt_1)

[[0.14567988]
 [0.04628124]
 [0.18182183]
 [0.59559035]
 [0.03062627]]


New code with new objective function

In [10]:
import cvxpy as cp
import numpy as np

w = cp.Variable((n, 1))
obj = cp.Minimize(cp.norm(C_3 - dts_t.T @ w, 2))
constraints = [w >= 0, w <= 0.03, cp.sum(w) == 1, Q_t.T @ w <= [[0.15, 0.05 , 0.20, 0.60, 0.03]]]
prob = cp.Problem(obj, constraints)
prob.solve()

print("status:", prob.status)
print("optimal value", prob.value)
w_opt = w.value
# print(Q_t.T @ w_opt)
print(Q_t.T @ w_opt)

 

status: optimal
optimal value 4.048217761010164e-09
[[0.13713726]
 [0.04947368]
 [0.19377632]
 [0.58982697]
 [0.02978578]]


Gradient of objective function @ w_opt

In [11]:
grad = -2 * (C_3 - w_opt.T @ dts_t) * dts_t
grad_sort = np.sort(grad, axis=0)[::-1][:10].round(20)
#selecting the non-zero values
grad_sort = grad[grad >= 0.00001]
print(grad_sort.shape)



(20,)
